In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [2]:
MAX_MEAT = 47
MAX_VEGGIE = 101
MAX_NOODLE = 268
BATCH_SIZE = 20
IMAGE_SIZE = (256,256)

In [3]:
file_name = ['filename'] + [str(i + 1) + '.jpg' for i in range(300)]
df = pd.DataFrame(file_name)
df.to_csv('test.csv', index = False, header = False)
df = pd.read_csv('./test.csv', header = 0, delimiter = ',')
df.head()

,filename
0,1.jpg
1,2.jpg
2,3.jpg
3,4.jpg
4,5.jpg


In [5]:
data_gen = ImageDataGenerator(rescale=1./255)

test_generator = data_gen.flow_from_dataframe(
    dataframe = df.loc[0 : 299],
    directory = './ranking_round/images',
    x_col = 'filename',
    y_col = [],
    shuffle = False,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
	class_mode= 'raw')

test_generator.reset()
print(len(test_generator))
model = load_model('model_best.h5')
predict = model.predict(
    test_generator,
    steps = len(test_generator),
    workers = 1,
    use_multiprocessing = False)

Found 300 validated image filenames.
15


In [20]:
print(len(predict))
result = []
test_generator.reset()
for (i, t) in enumerate(test_generator.labels) :
	file_name = str(i + 1) + '.jpg'
	meat, veggie, noodle = (predict[i][0] * MAX_MEAT, predict[i][1] * MAX_VEGGIE, predict[i][2] * MAX_NOODLE)
	result.append([file_name, meat, veggie, noodle])

df = pd.DataFrame(result)
df.head()

df.to_csv('result.csv', sep = ',', index = False, header = ['filename', 'meat', 'veggie', 'noodle'])

300
